In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pysyncon import Dataprep, Synth


In [ ]:
df = pd.read_csv('final_merged_data.csv')

In [ ]:



df['Country'] = df['Country'].replace({'Egypt, Arab Rep.': 'Egypt', 'Viet Nam': 'Vietnam'})


df = df.rename(columns={'International_Tourism_Receipts_Billions': 'Tourism_Receipts'})


df = df.sort_values(by=['Country', 'Year'])


df['Air_connectivity_proxy'] = df['Air_connectivity_proxy'].astype(str).str.replace(',', '', regex=False)
df['Air_connectivity_proxy'] = pd.to_numeric(df['Air_connectivity_proxy'], errors='coerce')
df['Air_connectivity_proxy'] = df.groupby('Country')['Air_connectivity_proxy'].ffill()


df['GDP_per_capita'] = df['GDP_per_capita'].astype(str).str.replace(',', '', regex=False)
df['GDP_per_capita'] = pd.to_numeric(df['GDP_per_capita'], errors='coerce')


df['Tourism_Receipts'] = pd.to_numeric(df['Tourism_Receipts'], errors='coerce')


print("DataFrame after re-initialization and cleaning:")
print(df.head())

In [ ]:
import numpy as np


df['Tourism_Receipts_2019_base'] = df.loc[df['Year'] == 2019].groupby('Country')['Tourism_Receipts'].transform('first')


df['Tourism_Receipts_2019_base'] = df.groupby('Country')['Tourism_Receipts_2019_base'].ffill().bfill()


df['Tourism_Receipts_Indexed'] = df.apply(lambda row: (row['Tourism_Receipts'] / row['Tourism_Receipts_2019_base']) * 100 if row['Tourism_Receipts_2019_base'] > 0 else 0, axis=1)


df['Tourism_Receipts_Indexed_log'] = np.log1p(df['Tourism_Receipts_Indexed'])


df['Tourism_Receipts_Indexed_log'] = df['Tourism_Receipts_Indexed_log'].round(2)


print("DataFrame head with original and new indexed/log-transformed columns:")
print(df[['Country', 'Year', 'Tourism_Receipts', 'Tourism_Receipts_2019_base', 'Tourism_Receipts_Indexed', 'Tourism_Receipts_Indexed_log']].head(10))


df = df.drop(columns=['Tourism_Receipts_2019_base', 'Tourism_Receipts_Indexed'])

print("\nDataFrame head after dropping intermediate columns:")
print(df.head())


In [ ]:
import numpy as np



df['Air_connectivity_proxy_log'] = np.log1p(df['Air_connectivity_proxy']).round(2)


df['GDP_per_capita_log'] = np.log1p(df['GDP_per_capita']).round(2)


donors = [c for c in df['Country'].unique() if c != 'India']


dataprep = Dataprep(
    foo=df,
    predictors=["GDP_per_capita_log", "Air_connectivity_proxy_log", "Inflation", "Trade", "StringencyIndex", "Exchange_Rate"],
    predictors_op="mean",
    time_predictors_prior=range(2014, 2022),
    dependent="Tourism_Receipts_Indexed_log",
    unit_variable="Country",
    time_variable="Year",
    treatment_identifier="India",
    controls_identifier=donors,
    time_optimize_ssr=range(2014, 2022),
    special_predictors=[
        ("Tourism_Receipts_Indexed_log", [2015], "mean"),
        ("Tourism_Receipts_Indexed_log", [2018], "mean"),
        ("Tourism_Receipts_Indexed_log", [2022], "mean"),
    ],
)


print("Dataprep object re-created successfully.")
print(f"Dependent variable: {dataprep.dependent}")
print(f"Predictors: {dataprep.predictors}")

In [ ]:
synth = Synth()
synth.fit(dataprep=dataprep)


print("Country Weights (W):")
print(synth.weights())

In [ ]:
import matplotlib.pyplot as plt


synth.path_plot(
    time_period=range(2014, 2025),
    treatment_time=2023,
    grid=True
)
plt.title("The G20 Effect: Real vs. Synthetic India (Indexed Tourism Receipts)")
plt.ylabel("Log of Indexed International Tourism Receipts")
plt.xlabel("Year")
plt.show()


synth.gaps_plot(
    time_period=range(2014, 2025),
    treatment_time=2023,
    grid=True
)
plt.title("Tourism Receipts Gap (Log Indexed Scale)")
plt.ylabel("Gap (Real - Synthetic)")
plt.xlabel("Year")
plt.show()

In [ ]:
from pysyncon.utils import PlaceboTest


placebo_test = PlaceboTest()
placebo_test.fit(
    dataprep=dataprep,
    scm=synth
)


p_val = placebo_test.pvalue(treatment_time=2023)

print(f"Empirical p-value: {p_val}")
if p_val < 0.10:
    print("The G20 effect is statistically significant at the 10% level.")
else:
    print("The effect is not statistically significant compared to donor volatility.")


placebo_test.gaps_plot(
    time_period=range(2014, 2025),
    treatment_time=2023,
    mspe_threshold=2 
)

In [ ]:
from pysyncon.utils import PlaceboTest


placebo_test = PlaceboTest()
placebo_test.fit(
    dataprep=dataprep,
    scm=synth
)


p_val = placebo_test.pvalue(treatment_time=2023)

print(f"Empirical p-value: {p_val}")
if p_val < 0.10:
    print("The G20 effect is statistically significant at the 10% level.")
else:
    print("The effect is not statistically significant compared to donor volatility.")


placebo_test.gaps_plot(
    time_period=range(2014, 2025),
    treatment_time=2023,
    mspe_threshold=2 
)

In [ ]:
import matplotlib.pyplot as plt


synth.path_plot(
    time_period=range(2014, 2025),
    treatment_time=2023,
    grid=False 
)


fig_path = plt.gcf()
ax_path = plt.gca()


if ax_path.lines:
    
    if len(ax_path.lines) > 0: 
        ax_path.lines[0].set_color('blue')
        ax_path.lines[0].set_label('India')
    if len(ax_path.lines) > 1: 
        ax_path.lines[1].set_color('red')
        ax_path.lines[1].set_label('Synthetic India') 

    
    for line in ax_path.lines:
        line.set_linewidth(2)
        line.set_linestyle('-')

    
    handles = [line for line in ax_path.lines if line.get_label() and not line.get_label().startswith('_')]
    labels = [line.get_label() for line in handles]

    
    ax_path.set_title("India vs. Synthetic India", fontsize=14, fontweight='bold', color='black')
    ax_path.set_ylabel("International Tourism Receipts (Indexed Log)", fontsize=12)
    ax_path.set_xlabel("Year", fontsize=12)
    ax_path.tick_params(axis='x', labelsize=10)
    ax_path.tick_params(axis='y', labelsize=10)
    ax_path.legend(handles=handles, labels=labels, fontsize=10)
    fig_path.set_size_inches(12, 7) 

    
    ax_path.grid(False)

  
    ax_path.spines['top'].set_visible(False)
    ax_path.spines['right'].set_visible(False)
    ax_path.spines['bottom'].set_visible(False)
    ax_path.spines['left'].set_visible(False)

    
    ax_path.tick_params(axis='x', colors='#CACCCE')
    ax_path.tick_params(axis='y', colors='#CACCCE')

    
    ax_path.axvline(x=2023, color='grey', linestyle='--', linewidth=1.5)
    ax_path.text(2023.1, ax_path.get_ylim()[1]*0.9, 'G20', rotation=90, va='top', ha='left', color='black', fontsize=10, fontweight='bold')

plt.show()


plt.figure()
synth.gaps_plot(
    time_period=range(2014, 2025),
    treatment_time=2023,
    grid=False 
)


fig_gaps = plt.gcf()
ax_gaps = plt.gca()


if ax_gaps.lines:
    for line in ax_gaps.lines:
        line.set_color('green')
        line.set_linewidth(2)

    
    ax_gaps.set_title("Gaps (Real - Synthetic India)", fontsize=14, fontweight='bold', color='black')
    ax_gaps.set_ylabel("Gap (Real - Synthetic)", fontsize=12)
    ax_gaps.set_xlabel("Year", fontsize=12)
    ax_gaps.tick_params(axis='x', labelsize=10)
    ax_gaps.tick_params(axis='y', labelsize=10)
    fig_gaps.set_size_inches(12, 7) 

    
    ax_gaps.grid(False)

    
    ax_gaps.spines['top'].set_visible(False)
    ax_gaps.spines['right'].set_visible(False)
    ax_gaps.spines['bottom'].set_visible(False)
    ax_gaps.spines['left'].set_visible(False)

   
    ax_gaps.tick_params(axis='x', colors='#CACCCE')
    ax_gaps.tick_params(axis='y', colors='#CACCCE')

    
    ax_gaps.axvline(x=2023, color='grey', linestyle='--', linewidth=1.5)
    ax_gaps.text(2023.1, ax_gaps.get_ylim()[1]*0.9, 'G20', rotation=90, va='top', ha='left', color='black', fontsize=10, fontweight='bold')

plt.show()

In [ ]:
import matplotlib.pyplot as plt


synth.path_plot(
    time_period=range(2014, 2025),
    treatment_time=2023,
    grid=False 
)


fig_path = plt.gcf()
ax_path = plt.gca()


if ax_path.lines:
    
    if len(ax_path.lines) > 0: 
        ax_path.lines[0].set_color('blue')
        ax_path.lines[0].set_label('India')
    if len(ax_path.lines) > 1: 
        ax_path.lines[1].set_color('red')
        ax_path.lines[1].set_label('Synthetic India') 

    
    for line in ax_path.lines:
        line.set_linewidth(2)
        line.set_linestyle('-')

    
    handles = [line for line in ax_path.lines if line.get_label() and not line.get_label().startswith('_')]
    labels = [line.get_label() for line in handles]

   
    ax_path.set_title("India vs. Synthetic India", fontsize=14, fontweight='bold', color='black')
    ax_path.set_ylabel("International Tourism Receipts (Indexed Log)", fontsize=12)
    ax_path.set_xlabel("Year", fontsize=12)
    ax_path.tick_params(axis='x', labelsize=10)
    ax_path.tick_params(axis='y', labelsize=10)
    ax_path.legend(handles=handles, labels=labels, fontsize=10)
    fig_path.set_size_inches(12, 7) 

    
    ax_path.grid(False)

   
    ax_path.spines['top'].set_visible(False)
    ax_path.spines['right'].set_visible(False)
    ax_path.spines['bottom'].set_visible(False)
    ax_path.spines['left'].set_visible(False)

    
    ax_path.tick_params(axis='x', colors='#CACCCE')
    ax_path.tick_params(axis='y', colors='#CACCCE')

    
    ax_path.axvline(x=2023, color='grey', linestyle='--', linewidth=1.5)
    ax_path.text(2023.1, ax_path.get_ylim()[1]*0.9, 'G20', rotation=90, va='top', ha='left', color='black', fontsize=10, fontweight='bold')

plt.show()


plt.figure()
synth.gaps_plot(
    time_period=range(2014, 2025),
    treatment_time=2023,
    grid=False 
)


fig_gaps = plt.gcf()
ax_gaps = plt.gca()


if ax_gaps.lines:
    for line in ax_gaps.lines:
        line.set_color('green')
        line.set_linewidth(2)

    
    ax_gaps.set_title("Gaps (Real - Synthetic India)", fontsize=14, fontweight='bold', color='black')
    ax_gaps.set_ylabel("Gap (Real - Synthetic)", fontsize=12)
    ax_gaps.set_xlabel("Year", fontsize=12)
    ax_gaps.tick_params(axis='x', labelsize=10)
    ax_gaps.tick_params(axis='y', labelsize=10)
    fig_gaps.set_size_inches(12, 7) 

    
    ax_gaps.grid(False)

    
    ax_gaps.spines['top'].set_visible(False)
    ax_gaps.spines['right'].set_visible(False)
    ax_gaps.spines['bottom'].set_visible(False)
    ax_gaps.spines['left'].set_visible(False)

    
    ax_gaps.tick_params(axis='x', colors='#CACCCE')
    ax_gaps.tick_params(axis='y', colors='#CACCCE')

    
    ax_gaps.axvline(x=2023, color='grey', linestyle='--', linewidth=1.5)
    ax_gaps.text(2023.1, ax_gaps.get_ylim()[1]*0.9, 'G20', rotation=90, va='top', ha='left', color='black', fontsize=10, fontweight='bold')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 


donor_weights = synth.weights().sort_values(ascending=False)
print("Donor Country Weights (sorted):")
print(donor_weights)


top_donors_count = 5 

active_donors = donor_weights[donor_weights > 0]
top_donors = active_donors.head(top_donors_count).index.tolist()

print(f"\nTop {top_donors_count} donor countries (with non-zero weights): {top_donors}")




india_actual_df = df[(df[dataprep.unit_variable] == dataprep.treatment_identifier) &
                     (df[dataprep.time_variable].isin(range(2014, 2025)))].set_index(dataprep.time_variable)[dataprep.dependent]


synthetic_india_values = []

active_donor_weights_dict = active_donors.to_dict()

for year in range(2014, 2025):
    
    donor_data_for_year = df[(df[dataprep.unit_variable].isin(active_donors.index.tolist())) &
                             (df[dataprep.time_variable] == year)]

    
    current_year_synthetic_value = 0
    for country in active_donors.index:
        country_data = donor_data_for_year[donor_data_for_year[dataprep.unit_variable] == country]
        if not country_data.empty:
            outcome_value = country_data[dataprep.dependent].iloc[0]
            current_year_synthetic_value += outcome_value * active_donor_weights_dict.get(country, 0)
    synthetic_india_values.append(current_year_synthetic_value)

synthetic_india_series = pd.Series(synthetic_india_values, index=pd.Index(range(2014, 2025), name='Year'))



donor_trajectories = df[
    (df[dataprep.unit_variable].isin(top_donors)) &
    (df[dataprep.time_variable].isin(range(2014, 2025)))
].pivot(index=dataprep.time_variable, columns=dataprep.unit_variable, values=dataprep.dependent)


plt.figure(figsize=(12, 7))


plt.plot(india_actual_df.index, india_actual_df, label='India (Real)', color='blue', linewidth=2)


plt.plot(synthetic_india_series.index, synthetic_india_series, label='Synthetic India', color='red', linestyle='--', linewidth=2)


for donor in top_donors:
    if donor in donor_trajectories.columns:
        plt.plot(donor_trajectories.index, donor_trajectories[donor], label=f'{donor}', linestyle=':', alpha=0.7)

plt.axvline(x=2023, color='grey', linestyle='--', linewidth=1.5)
plt.text(2023.1, plt.ylim()[1]*0.9, 'G20', rotation=90, va='top', ha='left', color='black', fontsize=10, fontweight='bold')

plt.title('Real India, Synthetic India, and Top Donor Countries Trajectories', fontsize=14, fontweight='bold', color='black')
plt.xlabel('Year', fontsize=12)
plt.ylabel('International Tourism Receipts (Indexed Log)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, linestyle=':', alpha=0.6)


ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', colors='#CACCCE', labelsize=10)
ax.tick_params(axis='y', colors='#CACCCE', labelsize=10)

plt.show()

In [ ]:
import numpy as np


india_2019_base_receipts = df[(df['Country'] == 'India') & (df['Year'] == 2019)]['Tourism_Receipts'].iloc[0]


india_actual_2023 = india_actual_df[2023]
synthetic_india_2023 = synthetic_india_series[2023]


actual_indexed_2023 = np.expm1(india_actual_2023) 
synthetic_indexed_2023 = np.expm1(synthetic_india_2023)



actual_dollar_2023 = (actual_indexed_2023 * india_2019_base_receipts) / 100
synthetic_dollar_2023 = (synthetic_indexed_2023 * india_2019_base_receipts) / 100


premium_dollar_2023 = actual_dollar_2023 - synthetic_dollar_2023

print(f"India's 2019 Base Tourism Receipts: ${india_2019_base_receipts:.2f} Billion")
print(f"\nIndia's Actual Tourism Receipts (2023): ${actual_dollar_2023:.2f} Billion")
print(f"Synthetic India's Tourism Receipts (2023): ${synthetic_dollar_2023:.2f} Billion")
print(f"Premium (Actual - Synthetic) for 2023: ${premium_dollar_2023:.2f} Billion")

In [ ]:
import numpy as np


calculation_year = 2024


india_2019_base_receipts = df[(df['Country'] == 'India') & (df['Year'] == 2019)]['Tourism_Receipts'].iloc[0]


india_actual_2024 = df[(df[dataprep.unit_variable] == dataprep.treatment_identifier) &
                       (df[dataprep.time_variable] == calculation_year)][dataprep.dependent].iloc[0]


donor_weights = synth.weights()
donors = donor_weights.index.tolist()


donor_outcomes_2024 = df[(df[dataprep.unit_variable].isin(donors)) &
                         (df[dataprep.time_variable] == calculation_year)].set_index(dataprep.unit_variable)[dataprep.dependent]


synthetic_india_2024 = (donor_outcomes_2024 * donor_weights[donor_outcomes_2024.index]).sum()


actual_indexed_2024 = np.expm1(india_actual_2024) 
synthetic_indexed_2024 = np.expm1(synthetic_india_2024)



actual_dollar_2024 = (actual_indexed_2024 * india_2019_base_receipts) / 100
synthetic_dollar_2024 = (synthetic_indexed_2024 * india_2019_base_receipts) / 100


premium_dollar_2024 = actual_dollar_2024 - synthetic_dollar_2024

print(f"India's 2019 Base Tourism Receipts: ${india_2019_base_receipts:.2f} Billion")
print(f"\nIndia's Actual Tourism Receipts ({calculation_year}): ${actual_dollar_2024:.2f} Billion")
print(f"Synthetic India's Tourism Receipts ({calculation_year}): ${synthetic_dollar_2024:.2f} Billion")
print(f"Premium (Actual - Synthetic) for {calculation_year}: ${premium_dollar_2024:.2f} Billion")


In [ ]:
import numpy as np


calculation_year = 2024


india_2019_base_receipts = df[(df['Country'] == 'India') & (df['Year'] == 2019)]['Tourism_Receipts'].iloc[0]


india_actual_2024 = df[(df[dataprep.unit_variable] == dataprep.treatment_identifier) &
                       (df[dataprep.time_variable] == calculation_year)][dataprep.dependent].iloc[0]


donor_weights = synth.weights()
donors = donor_weights.index.tolist()


donor_outcomes_2024 = df[(df[dataprep.unit_variable].isin(donors)) &
                         (df[dataprep.time_variable] == calculation_year)].set_index(dataprep.unit_variable)[dataprep.dependent]


synthetic_india_2024 = (donor_outcomes_2024 * donor_weights[donor_outcomes_2024.index]).sum()


actual_indexed_2024 = np.expm1(india_actual_2024) 
synthetic_indexed_2024 = np.expm1(synthetic_india_2024)


actual_dollar_2024 = (actual_indexed_2024 * india_2019_base_receipts) / 100
synthetic_dollar_2024 = (synthetic_indexed_2024 * india_2019_base_receipts) / 100


premium_dollar_2024 = actual_dollar_2024 - synthetic_dollar_2024

print(f"India's 2019 Base Tourism Receipts: ${india_2019_base_receipts:.2f} Billion")
print(f"\nIndia's Actual Tourism Receipts ({calculation_year}): ${actual_dollar_2024:.2f} Billion")
print(f"Synthetic India's Tourism Receipts ({calculation_year}): ${synthetic_dollar_2024:.2f} Billion")
print(f"Premium (Actual - Synthetic) for {calculation_year}: ${premium_dollar_2024:.2f} Billion")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


years = ['2023 (Summit Year)', '2024 (Lagged Effect)']
actual_receipts = [32.18, 34.89]    
synthetic_receipts = [31.58, 33.31] 
premiums = [0.60, 1.58]             

x = np.arange(len(years))  
width = 0.35               

fig, ax = plt.subplots(figsize=(10, 6))


rects1 = ax.bar(x - width/2, actual_receipts, width, label='Actual India', color='#003f5c')
rects2 = ax.bar(x + width/2, synthetic_receipts, width, label='Synthetic India (Counterfactual)', color='#ffa600')


ax.set_ylabel('Tourism Receipts (USD Billions)', fontsize=12, fontweight='bold')
ax.set_title('The G20 Impact: Actual vs. Synthetic Tourism Receipts', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(years, fontsize=11)
ax.legend()


def autolabel_bars(bars, offset_y=5):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'${height:.2f}B',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, offset_y),  
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10, color='black')

autolabel_bars(rects1, offset_y=5) 
autolabel_bars(rects2, offset_y=5) 


for i in range(len(years)):
    gap = actual_receipts[i] - synthetic_receipts[i]
    
    ax.annotate(f'+${gap:.2f}B Surplus',
                xy=(x[i], actual_receipts[i]), 
                xytext=(0, 18),  
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=11, fontweight='bold', color='green')


ax.grid(False)


ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)


ax.tick_params(axis='x', colors='black') 
ax.tick_params(axis='y', colors='#CACCCE') 


active_donors_names = active_donors.index.tolist() if 'active_donors' in locals() else ['Brazil', 'Philippines', 'Mexico', 'Egypt', 'Indonesia'] # Fallback
footnote_text = f"Note: Synthetic India constructed using weighted donor pool ({', '.join(active_donors_names).replace(', and', ' and')})."
plt.figtext(0.15, 0.01, footnote_text,
            ha="left", fontsize=9, style='italic') 

plt.tight_layout(rect=[0, 0.1, 1, 1]) 
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


years = ['2023 (Summit Year)', '2024 (Lagged Effect)']
actual_receipts = [32.18, 34.89]    
synthetic_receipts = [31.58, 33.31] 
premiums = [0.60, 1.58]             

x = np.arange(len(years))  
width = 0.35              

fig, ax = plt.subplots(figsize=(10, 6))


rects1 = ax.bar(x - width/2, actual_receipts, width, label='Actual India', color='#003f5c')
rects2 = ax.bar(x + width/2, synthetic_receipts, width, label='Synthetic India (Counterfactual)', color='#ffa600')


ax.set_ylabel('Tourism Receipts (USD Billions)', fontsize=12, fontweight='bold')
ax.set_title('The G20 Impact: Actual vs. Synthetic Tourism Receipts', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(years, fontsize=11)
ax.legend()


def autolabel_bars(bars, offset_y=5):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'${height:.2f}B',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, offset_y),  
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10, color='black')

autolabel_bars(rects1, offset_y=5) 
autolabel_bars(rects2, offset_y=5) 


for i in range(len(years)):
    gap = actual_receipts[i] - synthetic_receipts[i]
    
    ax.annotate(f'+${gap:.2f}B Surplus',
                xy=(x[i], actual_receipts[i]), 
                xytext=(0, 18),  
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=11, fontweight='bold', color='green')


ax.grid(False)


ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)


ax.tick_params(axis='x', colors='black') 
ax.tick_params(axis='y', colors='#CACCCE') 


active_donors_names = active_donors.index.tolist() if 'active_donors' in locals() else ['Brazil', 'Philippines', 'Mexico', 'Egypt', 'Indonesia'] # Fallback
footnote_text = f"Note: Synthetic India constructed using weighted donor pool ({', '.join(active_donors_names).replace(', and', ' and')})."
plt.figtext(0.15, 0.01, footnote_text,
            ha="left", fontsize=9, style='italic') 

plt.tight_layout(rect=[0, 0.1, 1, 1]) 
plt.show()
